In [3]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import tensorflow as tf
import sklearn
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle
from keras.callbacks import ModelCheckpoint
import matplotlib as mpl
import matplotlib.pyplot as plt
from glob import glob
import keras 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten


pickle_in = open("X.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

pickle_in = open("X_test.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("y_test.pickle","rb")
y_test = pickle.load(pickle_in)



y_train = keras.utils.to_categorical(y, num_classes=3)
y_test = keras.utils.to_categorical(y_test, num_classes=3)


cvacc =[]
cvAUC=[]
from sklearn.model_selection import KFold

kf=KFold(n_splits=10, random_state=10, shuffle=False)
for train_index, test_index, in kf.split(X_train):
    print("%s %s" % (train_index, test_index))
    X_train_c, X_test_c = X_train[train_index], X_train[test_index]
    y_train_c, y_test_c = y_train[train_index], y_train[test_index]
    reg=keras.regularizers.l2(0.0005)

    model = Sequential()

    model.add(Conv2D(560, (3, 3), activation ='relu', input_shape=(200, 200, 3), kernel_regularizer=reg, 
                     bias_regularizer=reg))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(280, (3, 3), activation ='relu', kernel_regularizer=reg, bias_regularizer=reg))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(140, (3, 3), activation ='relu', kernel_regularizer=reg, bias_regularizer=reg))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(70, (3, 3), activation ='relu', kernel_regularizer=reg, bias_regularizer=reg))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3), strides = 1, activation ='relu', kernel_regularizer=reg, bias_regularizer=reg))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(32, activation ='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(3, activation ='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

    
    model.load_weights('weights_cnn_model2_6.h5')
    
    cb= ModelCheckpoint(filepath='weights_cnn_best_crossvalidation.h5',  verbose=1, monitor='val_acc', 
                        save_best_only=True, save_weights_only=True)
     #validation_split=0.1,
    history=model.fit(X_train_c, y_train_c, batch_size=10, epochs=50, callbacks=[cb])
    
    '''print(history.history.keys())

    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    score, acc=model.evaluate( X_test_c, y_test_c)
    print('test loss')
    print(score)
    print ('test accuracy')
    print(acc)
    cvacc.append(acc)
    
    y_pred = model.predict_proba(X_test_c)
    auc_value=sklearn.metrics.roc_auc_score(y_test_c, y_pred)
    print ('AUC_ROC')
    print(auc_value)
    cvAUC.append(auc_value)

    # Plot of a ROC curve for a specific class
    
    def plot_ROC(y_test,y_pred):
    # Compute ROC curve and ROC area for each class
   
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(3):
            fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            print(roc_auc)
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        # Plot of a ROC curve for a specific class
        for i in range(3):
            plt.figure()
            plt.plot(fpr[i], tpr[i], label='ROC curve (area = {:.2f})'.format(roc_auc[i]))
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver operating characteristic example')
            plt.legend(loc="lower right")
            plt.show()


        from sklearn.metrics import confusion_matrix
    import itertools 
    cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    print('confusion matrix of : ',cm)

    def plot_confusion_matrix(cm, classes,
                             normalize=False,
                             title='confusion matrix',
                             cmap=plt.cm.Reds):
        # this funcation print & plots confusion matrix
        # normalize can be applied by  normalize=True,
        plt.figure()
        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title)
        plt.colorbar()
        tick_marks= np.arange(len(classes))
        plt.xticks(tick_marks, classes, rotation=45)
        plt.yticks(tick_marks, classes)

        if normalize:
            cm= cm.astype('float')/ cm.sum(axis=1)[:, np.newaxis]
            print('normalized confusion matrix')
        else:
            print(' confusion matrix, without normalized')
        print(cm)
        thresh=cm.max()/2.
        for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, cm[i,j],
                    horizontalalignment='center',
                    color='white' if cm[i, j] > thresh else 'black')

        plt.tight_layout()
        plt.ylabel('true label')
        plt.xlabel('predicted label')

        input to confusion_matrix must be a list of predictions, not OHEs (one hot encodings). 
        Call argmax on your y_test and y_pred, and you should get what you expect.

        from sklearn.utils.multiclass import unique_labels
        classes = [1,2,3]
        plot_confusion_matrix(cm,classes)'''
        

score, acc=model.evaluate( X_test_c, y_test_c)
print('test loss')
print(score)
print ('test accuracy')
print(acc)
cvacc.append(acc)
    
y_pred = model.predict_proba(X_test_c)
auc_value=sklearn.metrics.roc_auc_score(y_test_c, y_pred)
print ('AUC_ROC')
print(auc_value)
cvAUC.append(auc_value)

[ 18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Epoch 1/50
162/162 [==============================] - 5s 32ms/step - loss: 4.2725 - acc: 0.6235
Epoch 2/50


/home/tasneem/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


162/162 [==============================] - 4s 24ms/step - loss: 3.0149 - acc: 0.5309
Epoch 3/50
162/162 [==============================] - 4s 24ms/step - loss: 1.6200 - acc: 0.6296
Epoch 4/50
162/162 [==============================] - 4s 24ms/step - loss: 1.0283 - acc: 0.6049
Epoch 5/50
162/162 [==============================] - 4s 24ms/step - loss: 0.9138 - acc: 0.6543
Epoch 6/50
162/162 [==============================] - 4s 24ms/step - loss: 0.8229 - acc: 0.6481
Epoch 7/50
162/162 [==============================] - 4s 24ms/step - loss: 0.8933 - acc: 0.6543
Epoch 8/50
162/162 [==============================] - 4s 24ms/step - loss: 0.6794 - acc: 0.7716
Epoch 9/50
162/162 [==============================] - 4s 24ms/step - loss: 0.7788 - acc: 0.6790
Epoch 10/50
162/162 [==============================] - 4s 24ms/step - loss: 0.7323 - acc: 0.7284
Epoch 11/50
162/162 [==============================] - 4s 24ms/step - loss: 0.6638 - acc: 0.7407
Epoch 12/50
162/162 [============================

162/162 [==============================] - 4s 24ms/step - loss: 0.4470 - acc: 0.8642
Epoch 30/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4763 - acc: 0.8642
Epoch 31/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4122 - acc: 0.8889
Epoch 32/50
162/162 [==============================] - 4s 24ms/step - loss: 0.3794 - acc: 0.8704
Epoch 33/50
162/162 [==============================] - 4s 24ms/step - loss: 0.5761 - acc: 0.8642
Epoch 34/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4977 - acc: 0.8457
Epoch 35/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4456 - acc: 0.8395
Epoch 36/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4672 - acc: 0.8704
Epoch 37/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4138 - acc: 0.8765
Epoch 38/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4902 - acc: 0.8642
Epoch 39/50
162/162 [=====================

Epoch 1/50
162/162 [==============================] - 6s 34ms/step - loss: 4.5489 - acc: 0.6296
Epoch 2/50
162/162 [==============================] - 4s 24ms/step - loss: 2.0778 - acc: 0.6173
Epoch 3/50
162/162 [==============================] - 4s 24ms/step - loss: 1.1715 - acc: 0.7654
Epoch 4/50
162/162 [==============================] - 4s 24ms/step - loss: 1.1905 - acc: 0.6790
Epoch 5/50
162/162 [==============================] - 4s 24ms/step - loss: 0.8405 - acc: 0.6975
Epoch 6/50
162/162 [==============================] - 4s 24ms/step - loss: 0.8116 - acc: 0.7099
Epoch 7/50
162/162 [==============================] - 4s 24ms/step - loss: 0.6810 - acc: 0.7407
Epoch 8/50
162/162 [==============================] - 4s 24ms/step - loss: 0.6379 - acc: 0.8086
Epoch 9/50
162/162 [==============================] - 4s 24ms/step - loss: 0.7386 - acc: 0.7531
Epoch 10/50
162/162 [==============================] - 4s 24ms/step - loss: 0.7594 - acc: 0.7222
Epoch 11/50
162/162 [==================

162/162 [==============================] - 4s 24ms/step - loss: 0.3861 - acc: 0.9012
Epoch 29/50
162/162 [==============================] - 4s 24ms/step - loss: 0.5644 - acc: 0.8272
Epoch 30/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4698 - acc: 0.8457
Epoch 31/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4244 - acc: 0.8333
Epoch 32/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4698 - acc: 0.8642
Epoch 33/50
162/162 [==============================] - 4s 24ms/step - loss: 0.5455 - acc: 0.8272
Epoch 34/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4441 - acc: 0.8889
Epoch 35/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4140 - acc: 0.8642
Epoch 36/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4174 - acc: 0.8704
Epoch 37/50
162/162 [==============================] - 4s 24ms/step - loss: 0.5343 - acc: 0.8519
Epoch 38/50
162/162 [=====================

Epoch 1/50
162/162 [==============================] - 6s 36ms/step - loss: 4.5277 - acc: 0.5679
Epoch 2/50
162/162 [==============================] - 4s 24ms/step - loss: 2.3920 - acc: 0.6605
Epoch 3/50
162/162 [==============================] - 4s 24ms/step - loss: 1.9998 - acc: 0.5617
Epoch 4/50
162/162 [==============================] - 4s 24ms/step - loss: 1.1734 - acc: 0.6481
Epoch 5/50
162/162 [==============================] - 4s 24ms/step - loss: 0.9228 - acc: 0.6605
Epoch 6/50
162/162 [==============================] - 4s 24ms/step - loss: 0.9344 - acc: 0.6543
Epoch 7/50
162/162 [==============================] - 4s 24ms/step - loss: 0.7810 - acc: 0.7284
Epoch 8/50
162/162 [==============================] - 4s 24ms/step - loss: 0.7856 - acc: 0.7346
Epoch 9/50
162/162 [==============================] - 4s 24ms/step - loss: 0.7401 - acc: 0.7346
Epoch 10/50
162/162 [==============================] - 4s 24ms/step - loss: 0.6797 - acc: 0.7346
Epoch 11/50
162/162 [==================

162/162 [==============================] - 4s 24ms/step - loss: 0.5275 - acc: 0.8519
Epoch 29/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4773 - acc: 0.8333
Epoch 30/50
162/162 [==============================] - 4s 24ms/step - loss: 0.3955 - acc: 0.8704
Epoch 31/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4519 - acc: 0.8333
Epoch 32/50
162/162 [==============================] - 4s 24ms/step - loss: 0.6013 - acc: 0.8210
Epoch 33/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4771 - acc: 0.8333
Epoch 34/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4533 - acc: 0.8765
Epoch 35/50
162/162 [==============================] - 4s 24ms/step - loss: 0.4763 - acc: 0.8086
Epoch 36/50
162/162 [==============================] - 4s 24ms/step - loss: 0.3418 - acc: 0.9012
Epoch 37/50
162/162 [==============================] - 4s 24ms/step - loss: 0.3689 - acc: 0.8765
Epoch 38/50
162/162 [=====================

Epoch 1/50
162/162 [==============================] - 6s 38ms/step - loss: 4.3005 - acc: 0.5988
Epoch 2/50
162/162 [==============================] - 4s 24ms/step - loss: 3.3922 - acc: 0.6420
Epoch 3/50
162/162 [==============================] - 4s 24ms/step - loss: 2.0155 - acc: 0.6358
Epoch 4/50
162/162 [==============================] - 4s 24ms/step - loss: 1.3911 - acc: 0.6358
Epoch 5/50
162/162 [==============================] - 4s 24ms/step - loss: 1.2387 - acc: 0.6235
Epoch 6/50
162/162 [==============================] - 4s 24ms/step - loss: 0.8942 - acc: 0.6481
Epoch 7/50
162/162 [==============================] - 4s 24ms/step - loss: 0.9473 - acc: 0.7284
Epoch 8/50
162/162 [==============================] - 4s 24ms/step - loss: 0.9443 - acc: 0.5988
Epoch 9/50
162/162 [==============================] - 4s 24ms/step - loss: 0.7342 - acc: 0.7407
Epoch 10/50
162/162 [==============================] - 4s 24ms/step - loss: 0.8505 - acc: 0.7284
Epoch 11/50
162/162 [==================

In [4]:
model.save('kfold_right_100acc_99.5auc.h5')

In [1]:
print('The mean accuracy')
print(np.mean(cvacc))

print('The mean AUC')
print(np.mean(cvAUC))



The mean accuracy


NameError: name 'np' is not defined